In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
coordinates

[(-38.955012565448385, 86.42523673873376),
 (31.690172903380628, 67.38462028664108),
 (12.234681618377465, 74.37950574214887),
 (78.91122047927561, -136.25174149211838),
 (-5.713904470804394, 5.70359720889013),
 (1.0749745164423103, 174.5171995425536),
 (56.563734837206624, 72.19091569669487),
 (37.52543773615092, 109.48348283377356),
 (17.35527710724334, -1.9485136388162232),
 (30.51428565442407, 39.84913971527146),
 (-25.789709872478, -44.326798274727395),
 (-39.21292140054861, 145.02788941462973),
 (-76.39694044673705, -113.11282302364661),
 (-77.77309972144941, 65.98172527741053),
 (40.479362619219216, 123.32329586666799),
 (64.30299764378896, 24.52160117014668),
 (47.253872203699956, 86.6985289361865),
 (32.561683609003126, 131.73510536436385),
 (32.821434578833404, 93.52684712343057),
 (33.241806229458675, -175.85643903229806),
 (37.80985760865241, 162.25738094362504),
 (58.66053685682061, -27.877011868281983),
 (3.7376777243307515, -155.95847045697158),
 (75.69727631593238, -89.

In [5]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-38.955012565448385 86.42523673873376
31.690172903380628 67.38462028664108
12.234681618377465 74.37950574214887
78.91122047927561 -136.25174149211838
-5.713904470804394 5.70359720889013
1.0749745164423103 174.5171995425536
56.563734837206624 72.19091569669487
37.52543773615092 109.48348283377356
17.35527710724334 -1.9485136388162232
30.51428565442407 39.84913971527146
-25.789709872478 -44.326798274727395
-39.21292140054861 145.02788941462973
-76.39694044673705 -113.11282302364661
-77.77309972144941 65.98172527741053
40.479362619219216 123.32329586666799
64.30299764378896 24.52160117014668
47.253872203699956 86.6985289361865
32.561683609003126 131.73510536436385
32.821434578833404 93.52684712343057
33.241806229458675 -175.85643903229806
37.80985760865241 162.25738094362504
58.66053685682061 -27.877011868281983
3.7376777243307515 -155.95847045697158
75.69727631593238 -89.86349023371204
-61.107533097815455 76.83093938740672
-10.636401420896718 134.27813002229783
25.7350203241936 91.398627

-77.69202646546069 -175.25290654235107
18.03739998256718 160.25401243435078
-27.028985692887183 -161.03708033857623
37.30419820811308 -68.9637702931885
-87.04488616216493 -81.27979227297773
21.164826868148396 -70.71576735017868
-46.208276637411 60.40152073397451
-3.032286685075192 -170.52767252515798
77.8517358073602 160.21066313024136
46.8245135240341 -38.63928852675974
-12.030959436020439 -143.85883762265706
-78.53946401173411 -168.59311367958858
-19.886303993772273 -140.93137380041793
85.3187038960634 -32.61423061391153
-25.11116621761299 -71.51976311616234
79.39230178061825 40.151940188509485
82.24091937089682 109.4156611177782
3.2895270166569617 144.05871404778554
72.46491389265967 -54.6082225483927
-17.4926343979038 131.34274858753236
-6.129575542175516 -43.53964910388174
28.41047562024835 97.578747103855
86.35139827756402 101.68802980190236
-71.81373995759729 143.2601270873414
69.74675920621848 9.838063884148823
-65.62340208664358 129.0549552667726
-3.7840184414628624 8.80855393

In [7]:
from citipy import citipy

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
# cities

596